In [1]:
import cv2

import pyrealsense2 as rs
import numpy as np

class DepthCamera:
    def __init__(self):
        # Configure depth and color streams
        self.pipeline = rs.pipeline()
        config = rs.config()

        # Get device product line for setting a supporting resolution
        pipeline_wrapper = rs.pipeline_wrapper(self.pipeline)
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))

        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)


        # Start streaming
        self.pipeline.start(config)

    def get_frame(self):
        frames = self.pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        if not depth_frame or not color_frame:
            return False, None, None
        return True, depth_image, color_image

    def release(self):
        self.pipeline.stop()

In [2]:
import numpy as np
import cv2
import os

model_path = os.getenv('HOME') + '/aiffel/darknet-master/yolov4.weights'
list_path = os.getenv('HOME') + '/aiffel/darknet-master/cfg/yolov4.cfg'
name_path = os.getenv('HOME') + '/aiffel/darknet-master/data/coco.names'

YOLO_net = cv2.dnn.readNet(model_path, list_path)

classes = []
with open(name_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = YOLO_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in YOLO_net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [6]:
# Initialize Camera Intel Realsense
dc = DepthCamera()

def show_loc(x, y):
    cv2.circle(color_frame, (x, y), 4, (0, 0, 255))
    
    distance = depth_frame[y, x]
    cv2.putText(color_frame, "{}mm".format(distance), (x, y + 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)

while True:
    # 웹캠 프레임
    ret, depth_frame, color_frame = dc.get_frame()
    h, w, c = color_frame.shape

    # YOLO 입력
    blob = cv2.dnn.blobFromImage(color_frame, 0.00392, (640, 480), (0, 0, 0), True, crop=False)
    YOLO_net.setInput(blob)
    outs = YOLO_net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:

        for detection in out:

            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * w)
                center_y = int(detection[1] * h)
                dw = int(detection[2] * w)
                dh = int(detection[3] * h)
                # Rectangle coordinate
                x = int(center_x - dw / 2)
                y = int(center_y - dh / 2)
                boxes.append([x, y, dw, dh])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.4)


    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]

            label = str(classes[class_ids[i]])
            score = confidences[i]
            
            print(label)
            
            if label == 'person':

                # 경계상자와 클래스 정보 이미지에 입력
                cv2.rectangle(color_frame, (x, y), (x + w, y + h), (0, 0, 255), 5)
                cv2.putText(color_frame, label, (x, y - 20), cv2.FONT_ITALIC, 0.5, (255, 255, 255), 1)
                
                ROI = depth_frame[y:y+h,x:x+w]
                
                
                print(min(ROI[ROI>0]))
                
                #if(len(ROI) != 0):
                #    print(min(ROI[ROI>0])
            
                c_x = x + w//2
                c_y = y + h//2
                show_loc(c_x, c_y)

    cv2.imshow("YOLOv4", color_frame)
    #cv2.imshow("YOLOv4_lenght", depth_frame)

    key = cv2.waitKey(1)
    if key == 27:
        dc.release()
        cv2.destroyAllWindows()
        break



cup
mouse
person
154
cup
mouse
person
154
cup
mouse
person
154
mouse
cup
mouse
mouse
person
154
keyboard
cup
mouse
mouse
person
154
keyboard
cup
mouse
mouse
person
154
cup
mouse
mouse
person
154
mouse
cup
mouse
person


ValueError: min() arg is an empty sequence